In [14]:
!pip install accelerate protobuf sentencepiece torch git+https://github.com/huggingface/transformers

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-gu09y79w
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-gu09y79w
  Resolved https://github.com/huggingface/transformers to commit f2c388e3f946862f657acc1e21b272ec946fc66c
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [15]:
!pip -q install openai pinecone-client gradio sentence-transformers

In [16]:
# API Keys from colab secret for Hugging Face, OpenAI, and Pinecone
from google.colab import userdata
pinecone_api_key = userdata.get('PINECONE_API_KEY')
huggingface_api_key = userdata.get('HUGGINGFACEHUB_API_TOKEN')
openai_api_key = userdata.get('OPENAI_API_KEY')
pinecone_region = userdata.get('PINECONE_ENV')

In [17]:
import pandas as pd
import os
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from huggingface_hub import login
import torch

In [18]:
# Hugging Face Authentication
login(token=huggingface_api_key)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [19]:
# Define the paths for the CSV files
products_csv = 'products.csv'
qa_csv = 'qa_dataset.csv'
troubleshooting_csv = 'troubleshooting.csv'



# Load the existing CSV file into a DataFrame
qa_df = pd.read_csv(qa_csv)
products_df = pd.read_csv(products_csv)
troubleshooting_df = pd.read_csv(troubleshooting_csv)

In [20]:
# Verify the CSV content
print("QA Dataset:")
print(qa_df.head())

print("\nProducts Dataset:")
print(products_df.head())

print("\nTroubleshooting Dataset:")
print(troubleshooting_df.head())

QA Dataset:
                                            Question  \
0  My SmartHome Hub won't connect to Wi-Fi. What ...   
1  The temperature readings on my Smart Thermosta...   
2  My Smart Lights won't turn on or off using the...   
3  The Smart Lock isn't responding to app command...   
4  My Smart Security Camera isn't showing a live ...   

                                              Answer  
0  I understand you're having trouble connecting ...  
1  I'm sorry to hear your Smart Thermostat is sho...  
2  I see you're having issues controlling your Sm...  
3  I understand your Smart Lock isn't responding ...  
4  I'm sorry to hear you're not getting a live fe...  

Products Dataset:
                           Title  \
0              SmartHome Hub Pro   
1      EcoTherm Smart Thermostat   
2      LuminaGlow Smart Bulb Set   
3         SecureGuard Smart Lock   
4  ClearView Pro Security Camera   

                                         Description  
0  Control your entire smart h

In [21]:
from pinecone import Pinecone, ServerlessSpec
import openai
from sentence_transformers import SentenceTransformer


# Load the sentence transformer model
embedding_model = SentenceTransformer('sentence-transformers/multi-qa-mpnet-base-cos-v1')

# Initialize Pinecone
pc = Pinecone(api_key=pinecone_api_key)

# Pinecone index name
index_name = "llama-quest-index"


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [22]:
# Check if the index exists, if not, create it
if index_name not in pc.list_indexes():
    pc.create_index(
        name=index_name,
        dimension=768,  # Embedding dimension for 'sentence-transformers/multi-qa-mpnet-base-cos-v1'
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",
            region=pinecone_region
        )
    )

In [23]:
# Connect to the Pinecone index
index = pc.Index(index_name)

In [24]:
# Initialize the Llama 2 model and tokenizer
model_id = "NousResearch/Llama-2-7b-chat-hf"
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.use_default_system_prompt = False

# Initialize the pipeline using Hugging Face pipeline
llama_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    device_map="auto",
    max_length=1024,
)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [27]:
# Embed and store documents in Pinecone
def embed_and_store_documents(df, text_column, namespace, metadata_columns=None):
    if metadata_columns is None:
        metadata_columns = []  # Ensure it's an empty list if not provided

    for idx, row in df.iterrows():
        # Handle text_column being a list or string
        text = row[text_column] if isinstance(text_column, str) else ' '.join([str(row[col]) for col in text_column])

        # Embed the text using the sentence-transformers model
        embedding = embedding_model.encode(text).tolist()  # Convert to list format for Pinecone

        # Extract metadata
        metadata = {col: row[col] for col in metadata_columns}  # Handle metadata columns

        # Insert into Pinecone with unique ID (index)
        index.upsert([(str(idx), embedding, metadata)], namespace=namespace)


In [28]:
# Function to load datasets and index them correctly
def index_datasets():
    # Load datasets
    qa_df = pd.read_csv('qa_dataset.csv')
    products_df = pd.read_csv('products.csv')
    troubleshooting_df = pd.read_csv('troubleshooting.csv')

    # Index QA dataset: Use "Answer" as document, "Question" as metadata
    embed_and_store_documents(qa_df, 'Answer', 'qa', metadata_columns=['Question'])

    # Index Products dataset: Use "Description" as document, "Title" as metadata
    embed_and_store_documents(products_df, 'Description', 'products', metadata_columns=['Title'])

    # Index Troubleshooting dataset: Concatenate "Issue" and "Steps" for document, "Device" as metadata
    embed_and_store_documents(troubleshooting_df, ['Issue', 'Steps'], 'troubleshooting', metadata_columns=['Device'])

# Run the indexing process
index_datasets()


In [29]:
# Semantic search function using Pinecone
def semantic_search(query, namespace):
    # Use sentence-transformers to encode the query
    query_embedding = embedding_model.encode(query).tolist()

    # Query Pinecone
    results = index.query(
        vector=query_embedding,  # The query embedding
        top_k=3,  # Top 3 results
        include_metadata=True,  # Include metadata in the results
        namespace=namespace  # Search in the specified namespace
    )

    return results

In [30]:
# Function to generate a response with context from the search results
def generate_with_context(question, search_results):
    # Prepare the context from search results
    if search_results:
        best_match = search_results[0]['metadata']  # Assuming metadata holds relevant information
        context = f"Context: {best_match}\n\n"
        full_prompt = f"{context}User Question: {question}\n\nAI Response:"
    else:
        # No search results, just use the question
        full_prompt = f"User Question: {question}\n\nAI Response:"

    # Generate response using Llama-2
    response = llama_pipeline(full_prompt, max_length=200, do_sample=True)[0]['generated_text']
    return response

In [ ]:
# Main function to handle a user's question and provide an answer
def answer_question(question, history):
    # Perform semantic search across multiple namespaces (QA, products, troubleshooting)
    search_results = []
    search_results.extend(semantic_search(question, 'qa').matches)
    search_results.extend(semantic_search(question, 'products').matches)
    search_results.extend(semantic_search(question, 'troubleshooting').matches)

    # Generate answer using the search results as context
    response = generate_with_context(question, search_results)
    return response

res = answer_question("Smart tv misbehaving", [])
res

In [34]:
import gradio as gr

# Gradio chat interface
def gradio_chat_interface(question, history):
    response = answer_question(question, history)
    return response

In [35]:
# Create a Gradio Interface
interface = gr.ChatInterface(fn=gradio_chat_interface)

In [36]:
# Launch the Gradio Interface
interface.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://f85f2638e270618caa.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [37]:
# res = answer_question("What is the price of the iPhone 12?", [])
# res